In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
!pip install gdown

In [3]:
!pip install transformers[sentencepiece]

In [4]:
!pip install datasets

In [5]:
!pip install transformers[torch]

In [6]:
from transformers import AutoTokenizer

model_name = "t5-base"

tokenizer = AutoTokenizer.from_pretrained(model_name)

/opt/conda/lib/python3.10/site-packages/transformers/models/t5/tokenization_t5_fast.py:155: FutureWarning: This tokenizer was incorrectly instantiated with a model max length of 512 which will be corrected in Transformers v5.
For now, this behavior is kept to avoid breaking backwards compatibility when padding/encoding with `truncation is True`.
- Be aware that you SHOULD NOT rely on t5-base automatically truncating your input to 512 when padding/encoding.
- If you want to encode/pad to sequences longer than 512 you can either instantiate this tokenizer with `model_max_length` or pass `max_length` when encoding/padding.
- To avoid this warning, please instantiate this tokenizer with `model_max_length` set to your preferred value.
  warnings.warn(


In [7]:
import gdown

gdown.download("https://drive.google.com/uc?id=1-8Oh5eLEifAK9DEvTwmMWcWxdNxm9XoN", "train.csv")

Downloading...
From (uriginal): https://drive.google.com/uc?id=1-8Oh5eLEifAK9DEvTwmMWcWxdNxm9XoN
From (redirected): https://drive.google.com/uc?id=1-8Oh5eLEifAK9DEvTwmMWcWxdNxm9XoN&confirm=t&uuid=066b3fc4-74fc-4a91-976a-8607be9e5521
To: /kaggle/working/train.csv
100%|██████████| 1.26G/1.26G [00:14<00:00, 88.3MB/s]


'train.csv'

In [8]:
gdown.download("https://drive.google.com/uc?id=1-D1fQPyazMCCW-TKPJKAgfkxYuMQoi5a", 'validation.csv')

Downloading...
From: https://drive.google.com/uc?id=1-D1fQPyazMCCW-TKPJKAgfkxYuMQoi5a
To: /kaggle/working/validation.csv
100%|██████████| 57.7M/57.7M [00:00<00:00, 67.1MB/s]


'validation.csv'

In [9]:
import pandas as pd

df = pd.read_csv("/kaggle/working/train.csv")

In [10]:
df = df.drop(['id'], axis=1)

In [11]:
from datasets import Dataset

dataset = Dataset.from_pandas(df.sample(15000))

In [12]:
def convert_to_features(batch):
  input_encodings = tokenizer(batch['article'], max_length=1024, truncation=True)
  
  with tokenizer.as_target_tokenizer():
    target_encodings = tokenizer(batch['highlights'], max_length=256, truncation=True)

  return {
      'input_ids': input_encodings['input_ids'],
      'attention_mask': input_encodings['attention_mask'],
      'labels': target_encodings['input_ids']
  }

In [13]:
dataset_pt = dataset.map(convert_to_features, batched=True)
dataset_pt.save_to_disk("train_dataset")

  0%|          | 0/15 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [14]:
df_val = pd.read_csv("/kaggle/working/validation.csv")
dataset_val = Dataset.from_pandas(df_val)

dataset_val_pt = dataset_val.map(convert_to_features, batched=True)
dataset_val_pt.save_to_disk("validation_dataset")

  0%|          | 0/14 [00:00<?, ?ba/s]

/opt/conda/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


In [15]:
from transformers import TrainingArguments, Trainer, DataCollatorForSeq2Seq, AutoModelForSeq2SeqLM 
import torch

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:98: UserWarning: unable to load libtensorflow_io_plugins.so: unable to open file: libtensorflow_io_plugins.so, from paths: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so']
caused by: ['/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/libtensorflow_io_plugins.so: undefined symbol: _ZN3tsl6StatusC1EN10tensorflow5error4CodeESt17basic_string_viewIcSt11char_traitsIcEENS_14SourceLocationE']
  warnings.warn(f"unable to load libtensorflow_io_plugins.so: {e}")
/opt/conda/lib/python3.10/site-packages/tensorflow_io/python/ops/__init__.py:104: UserWarning: file system plugins are not loaded: unable to open file: l

In [16]:
device = "cuda" if torch.cuda.is_available() else "cpu"

model_t5 = AutoModelForSeq2SeqLM.from_pretrained(model_name)
seq2seq_data_collator = DataCollatorForSeq2Seq(tokenizer, model=model_t5)

trainer_args = TrainingArguments(
    output_dir="/content/drive/MyDrive/data/data_summarize/", num_train_epochs=1, warmup_steps=500,
    per_device_train_batch_size=1, per_device_eval_batch_size=1, weight_decay=0.01, logging_steps=10,
    evaluation_strategy='steps', eval_steps=500, save_steps=1e6,
    gradient_accumulation_steps=16
)

trainer = Trainer(model=model_t5.to(device), args=trainer_args,
                  tokenizer=tokenizer, data_collator=seq2seq_data_collator,
                  train_dataset=dataset_pt,
                  eval_dataset=dataset_val_pt)

In [17]:
trainer.train()

/opt/conda/lib/python3.10/site-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

  ········································


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


You're using a T5TokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Step,Training Loss,Validation Loss
500,1.453700,1.476254


TrainOutput(global_step=937, training_loss=1.6102212857029607, metrics={'train_runtime': 3356.8581, 'train_samples_per_second': 4.468, 'train_steps_per_second': 0.279, 'total_flos': 1.441484917797888e+16, 'train_loss': 1.6102212857029607, 'epoch': 1.0})

In [18]:
model_t5.save_pretrained("model_t5")

In [19]:
tokenizer.save_pretrained("tokenizer_t5")

('tokenizer_t5/tokenizer_config.json',
 'tokenizer_t5/special_tokens_map.json',
 'tokenizer_t5/spiece.model',
 'tokenizer_t5/added_tokens.json',
 'tokenizer_t5/tokenizer.json')

In [22]:
import pickle

with open("model", 'wb') as file:
    pickle.dump(model_t5, file)

In [23]:
with open('tokenizer', 'wb') as file:
    pickle.dump(tokenizer, file)

In [24]:
from IPython.display import FileLink
FileLink('model')

/kaggle/working/model

In [25]:
FileLink('tokenizer')

/kaggle/working/tokenizer